In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
import os 
from scipy.sparse import linalg
from pathlib import Path
import itertools
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu"    

torch.set_default_dtype(torch.float64)
pi = torch.tensor(np.pi)
ZERO = torch.tensor([0.]).to(device)
torch.set_printoptions(precision=6)

class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1
    def evaluate_derivative(self, x, i):
        if self.k == 1:
            u1 = self.fc2(torch.heaviside(self.fc1(x),ZERO) * self.fc1.weight.t()[i-1:i,:] )
        else:
            u1 = self.fc2(self.k*F.relu(self.fc1(x))**(self.k-1) *self.fc1.weight.t()[i-1:i,:] )  
        return u1


In [2]:
def show_convergence_order2(err_l2,err_h10,exponent,dict_size, filename,write2file = False):
    
    if write2file:
        file_mode = "a" if os.path.exists(filename) else "w"
        f_write = open(filename, file_mode)
    
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    err_list2 = [err_h10[i] for i in neuron_nums ] 
    # f_write.write('M:{}, relu {} \n'.format(M,k))
    if write2file:
        f_write.write('dictionary size: {}\n'.format(dict_size))
        f_write.write("neuron num \t\t error \t\t order \t\t h10 error \\ order \n")
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t")
            
            # print("*")
            print("{} \t\t {:.6f} \t\t * \t\t {:.6f} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t * \t\t {} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ))
        else: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t") 
            # print(np.log(err_list[i-1]/err_list[i])/np.log(2))
            print("{} \t\t {:.6f} \t\t {:.6f} \t\t {:.6f} \t\t {:.6f} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t {} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ))
    if write2file:     
        f_write.write("\n")
        f_write.close()

def show_convergence_order_latex2(err_l2,err_h10,exponent,k=1,d=1,m =1): 
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    err_list2 = [err_h10[i] for i in neuron_nums ] 
    l2_order = -1/2-(2*k + 1)/(2*d)
    h1_order =  -1/2-(2*(k-m)+ 1)/(2*d)
    print(" $n$  & \t $\|u-u_n \|_{{L^2}}$ & \t order $O(n^{{{:.2f}}})$  & \t $ | u -u_n |_{{H^1}}$ & \t order $O(n^{{{:.2f}}})$  \\\ \hline \hline ".format(l2_order,h1_order))
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t & {:.6f} &\t\t * & \t\t {:.6f} & \t\t *  \\\ \hline  \n".format(neuron_nums[i],item, err_list2[i] ) )   
        else: 
            print("{} \t\t &  {:.2e} &  \t\t {:.2f} &  \t\t {:.2e} &  \t\t {:.2f} \\\ \hline  \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ) )


### 2D example 1    

In [4]:
function_name = "biharmonic" 
relu_k= 3 
trial_num = 5
errl2_trials = [] 
errh2_trials = [] 
for trial in range(trial_num): 
    for N_list in [[2**5, 2**5]]: #[2**6,2**6],[2**7,2**7] 
        # save = True 
        my_model = None 
        Nx = 400
        order = 2   
        exponent = 9
        num_epochs = 2**exponent  
        N = np.prod(N_list)
        folder = './'
        filename = folder + 'err_OGA_2D_{}_neuron_{}_N_{}_rand_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        errl2 = torch.load(filename)
        filename = folder + 'errh2_OGA_2D_{}_neuron_{}_N_{}_rand_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        errh2 = torch.load(filename) 
        errl2_trials.append(errl2)
        errh2_trials.append(errh2)
errl2_trials = torch.stack(errl2_trials, dim = 0)
errh2_trials = torch.stack(errh2_trials,dim =0)
errl2_ave = errl2_trials.mean(dim = 0)
errh2_ave = errh2_trials.mean(dim=0)
errl2_var = errl2_trials.var(dim = 0)
errh2_var = errh2_trials.var(dim = 0)


show_convergence_order_latex2(errl2_ave, errh2_ave, exponent, k=3, d=2,m=2)


 $n$  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-2.25})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.25})$  \\ \hline \hline 
4 		 & 0.441577 &		 * & 		 15.589511 & 		 *  \\ \hline  

8 		 &  9.68e-01 &  		 -1.13 &  		 1.42e+01 &  		 0.14 \\ \hline  

16 		 &  9.78e-01 &  		 -0.01 &  		 9.45e+00 &  		 0.59 \\ \hline  

32 		 &  2.32e-02 &  		 5.40 &  		 3.75e+00 &  		 1.33 \\ \hline  

64 		 &  2.88e-03 &  		 3.01 &  		 1.42e+00 &  		 1.40 \\ \hline  

128 		 &  3.95e-04 &  		 2.87 &  		 5.12e-01 &  		 1.47 \\ \hline  

256 		 &  5.88e-05 &  		 2.75 &  		 2.05e-01 &  		 1.32 \\ \hline  

512 		 &  1.07e-05 &  		 2.46 &  		 7.90e-02 &  		 1.38 \\ \hline  



### 3D example 2 

In [15]:
dim = 3 
function_name = "cospix-osci-coef" 
trial_num = 5
errl2_trials = [] 
errh10_trials = [] 
relu_k = 3 
for N_list in [[2**3,2**3,2**3]]: 
    for trial in range(trial_num):
        exponent = 10 
        num_epochs = 2**exponent  
        N = np.prod(N_list)
        folder = './' 
        filename = folder + 'errl2_NeumannOGA_OsciCoeff_3D_{}_relu{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,relu_k, num_epochs,N,trial)
        errl2 = torch.load(filename)
        errl2_trials.append(errl2)

        filename = folder + 'errh10_NeumannOGA_OsciCoeff_3D_{}_relu{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,relu_k, num_epochs,N,trial)
        errh10 = torch.load(filename)
        errh10_trials.append(errh10)
        show_convergence_order_latex2(errl2, errh10, exponent, k=3, d=3)
errl2_trials = torch.stack(errl2_trials, dim = 0)
errh10_trials = torch.stack(errh10_trials,dim =0)
errl2_ave = errl2_trials.mean(dim = 0)
errh10_ave = errh10_trials.mean(dim=0)
errl2_var = errl2_trials.var(dim = 0)
errh10_var = errh10_trials.var(dim = 0)

# print(errl2_ave)
# print(errh10_ave)
# print(errl2_var)
# print(errh10_var)
show_convergence_order_latex2(errl2_ave, errh10_ave, exponent, k=relu_k, d=3)


neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.67})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.33})$  \\ \hline \hline 
4 		 & 0.348097 &		 * & 		 1.893182 & 		 *  \\ \hline  

8 		 &  2.33e-01 &  		 0.58 &  		 1.44e+00 &  		 0.39 \\ \hline  

16 		 &  9.35e-02 &  		 1.32 &  		 8.79e-01 &  		 0.72 \\ \hline  

32 		 &  2.95e-02 &  		 1.67 &  		 3.63e-01 &  		 1.27 \\ \hline  

64 		 &  6.87e-03 &  		 2.10 &  		 1.03e-01 &  		 1.83 \\ \hline  

128 		 &  1.22e-03 &  		 2.49 &  		 2.62e-02 &  		 1.97 \\ \hline  

256 		 &  3.01e-04 &  		 2.02 &  		 8.69e-03 &  		 1.59 \\ \hline  

512 		 &  8.33e-05 &  		 1.86 &  		 3.07e-03 &  		 1.50 \\ \hline  

1024 		 &  2.11e-05 &  		 1.98 &  		 1.03e-03 &  		 1.58 \\ \hline  

neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.67})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.33})$  \\ \hline \hline 
4 		 & 0.351756 &		 * & 		 1.895410 & 		 *  \\ \hline  

8 		 &  3.08e-01 &  		 0.19 &  		 1.46e+00 &  		 0.38 \\ \hline  

16 		 &  9.67

## 4D example 1 

In [8]:
function_name = "gaussian" 
trial_num = 5
errl2_trials = [] 
errh10_trials = [] 
for N_list in [[2**4,2**3,2**3]]: 
    for trial in range(trial_num):
        exponent = 10 
        num_epochs = 2**exponent  
        N = np.prod(N_list)
        folder = './' 
        filename = folder + 'errl2_OGA_4D_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        errl2 = torch.load(filename)
        errl2_trials.append(errl2)

        filename = folder + 'errh10_OGA_4D_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        errh10 = torch.load(filename)
        errh10_trials.append(errh10)
        show_convergence_order_latex2(errl2, errh10, exponent, k=3, d=3)
errl2_trials = torch.stack(errl2_trials, dim = 0)
errh10_trials = torch.stack(errh10_trials,dim =0)
errl2_ave = errl2_trials.mean(dim = 0)
errh10_ave = errh10_trials.mean(dim=0)
errl2_var = errl2_trials.var(dim = 0)
errh10_var = errh10_trials.var(dim = 0)

# print(errl2_ave)
# print(errh10_ave)
# print(errl2_var)
# print(errh10_var)
show_convergence_order_latex2(errl2_ave, errh10_ave, exponent, k=4, d=4)


neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.67})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.33})$  \\ \hline \hline 
4 		 & 0.162245 &		 * & 		 1.041883 & 		 *  \\ \hline  

8 		 &  1.083e-01 &  		 0.58 &  		 7.401e-01 &  		 0.49 \\ \hline  

16 		 &  7.134e-02 &  		 0.60 &  		 5.967e-01 &  		 0.31 \\ \hline  

32 		 &  3.687e-02 &  		 0.95 &  		 3.351e-01 &  		 0.83 \\ \hline  

64 		 &  1.037e-02 &  		 1.83 &  		 1.267e-01 &  		 1.40 \\ \hline  

128 		 &  4.489e-03 &  		 1.21 &  		 5.932e-02 &  		 1.10 \\ \hline  

256 		 &  7.315e-04 &  		 2.62 &  		 1.213e-02 &  		 2.29 \\ \hline  

512 		 &  1.521e-04 &  		 2.27 &  		 3.211e-03 &  		 1.92 \\ \hline  

1024 		 &  1.406e-04 &  		 0.11 &  		 3.953e-03 &  		 -0.30 \\ \hline  

neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.67})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.33})$  \\ \hline \hline 
4 		 & 0.159982 &		 * & 		 1.017259 & 		 *  \\ \hline  

8 		 &  1.117e-01 &  		 0.52 &  		 6.616e-01 &  		 0.62 \\ \hli

## 10D example 

In [9]:
function_name = "gaussian" 
trial_num = 5
errl2_trials = [] 
errh10_trials = []
relu_k = 4 
dim = 10  
for N_list in [[2**4,2**3,2**3]]: 
    for trial in range(trial_num):
        exponent = 10
        num_epochs = 2**exponent  
        N = np.prod(N_list)
        folder = './' 
        filename = folder + 'errl2_OGA_10D_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        errl2 = torch.load(filename)
        errl2_trials.append(errl2)

        filename = folder + 'errh10_OGA_10D_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        errh10 = torch.load(filename)
        errh10_trials.append(errh10)
        show_convergence_order_latex2(errl2, errh10, exponent, k=relu_k, d=dim)
errl2_trials = torch.stack(errl2_trials, dim = 0)
errh10_trials = torch.stack(errh10_trials,dim =0)
errl2_ave = errl2_trials.mean(dim = 0)
errh10_ave = errh10_trials.mean(dim=0)
errl2_var = errl2_trials.var(dim = 0)
errh10_var = errh10_trials.var(dim = 0)

# print(errl2_ave)
# print(errh10_ave)
# print(errl2_var)
# print(errh10_var)
show_convergence_order_latex2(errl2_ave, errh10_ave, exponent, k=relu_k, d=dim)


neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-0.95})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-0.85})$  \\ \hline \hline 
4 		 & 0.497536 &		 * & 		 0.649613 & 		 *  \\ \hline  

8 		 &  3.743e-01 &  		 0.41 &  		 6.339e-01 &  		 0.04 \\ \hline  

16 		 &  2.616e-01 &  		 0.52 &  		 5.192e-01 &  		 0.29 \\ \hline  

32 		 &  1.326e-01 &  		 0.98 &  		 3.149e-01 &  		 0.72 \\ \hline  

64 		 &  4.197e-02 &  		 1.66 &  		 2.174e-01 &  		 0.53 \\ \hline  

128 		 &  1.711e-02 &  		 1.29 &  		 1.260e-01 &  		 0.79 \\ \hline  

256 		 &  7.051e-03 &  		 1.28 &  		 6.089e-02 &  		 1.05 \\ \hline  

512 		 &  1.600e-03 &  		 2.14 &  		 1.403e-02 &  		 2.12 \\ \hline  

1024 		 &  1.481e-03 &  		 0.11 &  		 1.407e-02 &  		 -0.00 \\ \hline  

neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-0.95})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-0.85})$  \\ \hline \hline 
4 		 & 0.516911 &		 * & 		 0.653433 & 		 *  \\ \hline  

8 		 &  3.548e-01 &  		 0.54 &  		 6.480e-01 &  		 0.01 \\ \hli

In [19]:
## neuron number 1024 

function_name = "gaussian" 
trial_num = 5
errl2_trials = [] 
errh10_trials = []
relu_k = 4 
dim = 10  
for N_list in [[2**4,2**3,2**3]]: 
    for trial in range(trial_num):
        exponent = 10 
        num_epochs = 2**exponent  
        N = np.prod(N_list)
        folder = './' 
        filename = folder + 'errl2_OGA_10D_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        errl2 = torch.load(filename)
        errl2_trials.append(errl2)

        filename = folder + 'errh10_OGA_10D_{}_neuron_{}_N_{}_randomized_trial_{}.pt'.format(function_name,num_epochs,N,trial)
        errh10 = torch.load(filename)
        errh10_trials.append(errh10)
        show_convergence_order_latex2(errl2, errh10, exponent, k=relu_k, d=dim)
errl2_trials = torch.stack(errl2_trials, dim = 0)
errh10_trials = torch.stack(errh10_trials,dim =0)
errl2_ave = errl2_trials.mean(dim = 0)
errh10_ave = errh10_trials.mean(dim=0)
errl2_var = errl2_trials.var(dim = 0)
errh10_var = errh10_trials.var(dim = 0)

# print(errl2_ave)
# print(errh10_ave)
# print(errl2_var)
# print(errh10_var)
show_convergence_order_latex2(errl2_ave, errh10_ave, exponent, k=relu_k, d=dim)


neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-0.95})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-0.85})$  \\ \hline \hline 
4 		 & 0.497784 &		 * & 		 2.087274 & 		 *  \\ \hline  

8 		 &  3.542e-01 &  		 0.49 &  		 2.077e+00 &  		 0.01 \\ \hline  

16 		 &  2.320e-01 &  		 0.61 &  		 1.600e+00 &  		 0.38 \\ \hline  

32 		 &  1.385e-01 &  		 0.74 &  		 1.020e+00 &  		 0.65 \\ \hline  

64 		 &  4.742e-02 &  		 1.55 &  		 6.994e-01 &  		 0.54 \\ \hline  

128 		 &  1.820e-02 &  		 1.38 &  		 4.246e-01 &  		 0.72 \\ \hline  

256 		 &  7.143e-03 &  		 1.35 &  		 1.987e-01 &  		 1.10 \\ \hline  

512 		 &  1.607e-03 &  		 2.15 &  		 4.433e-02 &  		 2.16 \\ \hline  

1024 		 &  1.438e-03 &  		 0.16 &  		 4.176e-02 &  		 0.09 \\ \hline  

neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-0.95})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-0.85})$  \\ \hline \hline 
4 		 & 0.539957 &		 * & 		 2.059384 & 		 *  \\ \hline  

8 		 &  3.932e-01 &  		 0.46 &  		 2.061e+00 &  		 -0.00 \\ \hli